# Upload Block VQE

I will use this file to write the runtime program and then to upload it to the server.

The structure of the file will be as follows:

1) define the circuits

    a) diagonal
    b) off diagonal
    
2) define the energy

    a) measure_pauli
    b) energy on block
    c) energy off block
    d) total energy
    
3) define the SPSA optimization

In [14]:
%%writefile runtime_group_vqe.py

from qiskit import QuantumCircuit, QuantumRegister,ClassicalRegister, execute
from qiskit import transpile
import numpy as np
import copy
from datetime import datetime

def Ui(cir_i,phi_i):  
    Q = len(phi_i)
    for q in range(0,Q):
        cir_i.ry(phi_i[q],q)
    for q in range(0,Q,4):
        cir_i.cx(q,q+1)
        if q + 3 < Q:
            cir_i.cx(q+3,q+2)
    for q in range(1,Q,4):
        cir_i.cx(q,q+1)
        if q + 3 < Q:
            cir_i.cx(q+3,q+2)
    return cir_i

def U(cir,phi_b):
    T = len(phi_b)
    for t in range(T):
        cir = Ui(cir,phi_b[t])
    return cir

def Ui_off(cir_i,phi_a, phi_b):  
    Q = len(phi_a)
    for q in range(0,Q):
        cir_i.ry(phi_a[q],q)
        cir_i.cx(Q,q)
        cir_i.ry(-(phi_b[q]-phi_a[q])/2,q)
        cir_i.cx(Q,q)
        cir_i.ry((phi_b[q]-phi_a[q])/2,q)
    for q in range(0,Q,4):
        cir_i.cx(q,q+1)
        if q + 3 < Q:
            cir_i.cx(q+3,q+2)
    for q in range(1,Q,4):
        cir_i.cx(q,q+1)
        if q + 3 < Q:
            cir_i.cx(q+3,q+2)
    return cir_i

def U_off(cir,phi_a,phi_b):
    Q = len(phi_b[0])
    T = len(phi_b)
    cir.h(Q)
    for t in range(T):
        cir = Ui_off(cir,phi_a[t],phi_b[t])
    return cir


from qiskit import Aer
from qiskit.visualization import *
from qiskit import quantum_info as qi

def convert_to_circ(p_label,psi0):
    #apply rotations
    psi = copy.deepcopy(psi0)
    pauli_qs = []
    Z_label = ''
    Q = len(p_label)
    for q,p in enumerate(p_label):
        if p == 'X':
            psi.ry(-np.pi/2,Q-1-q)
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'Y':
            psi.rx(np.pi/2,Q-1-q)
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'Z':
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'I':
            pauli_qs.append(0)
            Z_label += 'I'       
    #apply measurment 
    psi.measure(psi.qubits,psi.clbits)
    #return circut and measurement instructions
    return [psi,pauli_qs]

#This collects the circuits for the on-diagonal blocks only
def collect_diag_circs(phi_b,block):
    Q = int(np.log2(len(block.to_matrix())))
    qr = QuantumRegister(Q)
    cr = ClassicalRegister(Q)
    cir = QuantumCircuit(qr , cr)
    psi0 = U(cir,phi_b)
    cirs = []
    meas = []
    coef = []
    for p in block:
        psi = copy.deepcopy(psi0)
        w = p.coeff
        if np.abs(w) > 0:
            p_label = p.primitive.to_label()
            cir_p,meas_p  = convert_to_circ(p_label,psi)
            cirs.append(cir_p)
            meas.append(meas_p)
            coef.append(w)
    return cirs,meas,coef

# This collects circuits for both on and off diagonal blocks but is unefficient for on-diagonal blocks
def collect_off_circs(phi_a,phi_b,block):
    Q = int(np.log2(len(block.to_matrix())))
    qr = QuantumRegister(Q+1)
    cr = ClassicalRegister(Q+1)
    cir = QuantumCircuit(qr , cr)
    psi0 = U_off(cir,phi_a,phi_b)
    cirs = []
    meas = []
    coef = []
    for p in block:
        psi = copy.deepcopy(psi0)
        w = p.coeff
        if np.abs(w) > 0.0:
            p_label = p.primitive.to_label()
            circ_x,meas_x = convert_to_circ('X' + p_label,psi)
            circ_y,meas_y = convert_to_circ('Y' + p_label,psi)
            cirs.append(circ_x)
            meas.append(meas_x)
            coef.append(w)
            cirs.append(circ_y)
            meas.append(meas_y)
            coef.append(1j*w)
    return cirs,meas,coef

def collect_circs(alpha,phi,blocks):
    circs = []
    meas = []
    coefs = []
    for key in list(blocks.keys()):
        if key[0] == key[2]:
            circ_b, meas_b, coef_b = collect_diag_circs(phi[int(key[0])],blocks[key])
            for b in range(len(coef_b)):
                circs.append(circ_b[b])
                meas.append(meas_b[b])
                coefs.append(alpha[int(key[0])]*alpha[int(key[2])]*coef_b[b])
        else:
            circ_b, meas_b, coef_b = collect_off_circs(phi[int(key[0])],phi[int(key[2])],blocks[key])
            for b in range(len(coef_b)):
                circs.append(circ_b[b])
                meas.append(meas_b[b])
                coefs.append(2*alpha[int(key[0])]*alpha[int(key[2])]*coef_b[b])
    return circs, meas, coefs

def get_results(backend,circs, method = 'matrix',save_id_file = None):
    if method == 'matrix':
        # add results
        r = []
        for l in range(len(circs)):
            circs[l].remove_final_measurements()
            wave = qi.Statevector.from_instruction(circs[l])
            r_l = wave.probabilities_dict()
            r.append(r_l)
    
    if method == "simulator":
        sim = Aer.get_backend("qasm_simulator")
        r = execute(circs, backend = sim).result().get_counts()
        
    if method == "quantum":
        job = backend.run(transpile(circs, backend), meas_level=2, shots=8192)
        job_id = job.job_id()
        print(job_id)
        np.save(save_id_file,[job_id])
        r = job.result().get_counts()
    return r

def energy_from_results(r,meas,coefs):
    E = 0
    for l in range(len(r)):
        z_measure = 0
        total = 0
        for key in list(r[l].keys()):
            n = 0
            for q in range(len(key)):
                if key[q] == '1' and meas[l][q] == 1:
                    n += 1
            z_measure += (-1)**n * r[l][key] 
            total += r[l][key]
        E += coefs[l]*z_measure/total
    return np.real(E)

def collect_energy_circs(a1, p1, b1, a2, p2, b2, a3, p3, b3):
    circs1, meas1, coefs1 = collect_circs(a1,p1,b1)
    circs2, meas2, coefs2 = collect_circs(a2,p2,b2)
    circs3, meas3, coefs3 = collect_circs(a3,p3,b3)
    L1 = len(circs1)
    L2 = len(circs2)
    L3 = len(circs3)
    circs = circs1+circs2+circs3
    meas = meas1+meas2+meas3
    coefs = coefs1+coefs2+coefs3
    lengths = [L1,L2,L3]
    return circs,meas,coefs,lengths

def three_energies_from_results(r,meas,coefs,lengths):
    L1 = lengths[0]
    L2 = lengths[1]
    L3 = lengths[2]
    r1 = r[0 : L1]
    r2 = r[L1 : L1+L2]
    r3 = r[L1+L2 : L1+L2+L3]
    m1 = meas[0 : L1]
    m2 = meas[L1 : L1+L2]
    m3 = meas[L1+L2 : L1+L2+L3]
    c1 = coefs[0 : L1]
    c2 = coefs[L1 : L1+L2]
    c3 = coefs[L1+L2 : L1+L2+L3]
    E1 = energy_from_results(r1,m1,c1)
    E2 = energy_from_results(r2,m2,c2)
    E3 = energy_from_results(r3,m3,c3)
    return E1,E2,E3
    
def SPSA(backend, user_messenger, k_max, tt, u, phi, alpha, blocks, method = 'simple', group = False ,hold = False):
    import copy
    beta = 0.201
    A = 10
    a = 0.05
    gamma = 0.101
    c = 0.4

    beta_a = 0.201
    A_a = 10
    a_a = 0.05
    gamma_a = 0.101
    c_a = 0.4
    
    
    #Initalization
    k = 0
    phi_k = np.array(phi)
    alpha_k = np.array(alpha)
    bL = len(phi_k)
    T = len(phi_k[0])
    Q = len(phi_k[0][0])
    E_l = []
    hold_l = []
    
    #Begin Iterations
    for k in range(k_max):
        #Update c and a
        a_k = a/((A + k + 1)**beta)
        c_k = c/((k + 1)**gamma)
        a_ak = a_a/((A_a + k + 1)**beta_a)
        c_ak = c_a/((k + 1)**gamma_a)

        #Find Delta
        Delta_k = np.array(phi_k)
        for b in range(bL):
            for t in range(T):
                for q in range(Q):
                    Delta_k[b][t][q] = 1 - 2*np.random.binomial(size=None, n=1, p=0.5)
        phi_k_A = phi_k + c_k*Delta_k
        phi_k_B = phi_k - c_k*Delta_k
        
        #Find Delta Alpha
        Delta_ak = np.array(alpha_k)
        for n in range(bL):
            Delta_ak[n] = 1 - 2*np.random.binomial(size=None, n=1, p= 0.5)
        alpha_k_A = alpha_k + c_ak*Delta_ak
        alpha_k_B = alpha_k - c_ak*Delta_ak
        norm_A = 1/np.sqrt(np.dot(alpha_k_A,alpha_k_A))
        norm_B = 1/np.sqrt(np.dot(alpha_k_B,alpha_k_B))
        alpha_k_A = norm_A*alpha_k_A
        alpha_k_B = norm_B*alpha_k_B
        
        user_messenger.publish({'starting iteration':k})
        #Find E    
        if group:
            save_id_file = "/Users/stenger/Documents/Research/Hubbard_symmetries/VQE/Job_id/job_"+str(k)+'_T_'+str(T)+"_A_"+str(A)+"_a_"+str(a)+"_c_"+str(c)+"_beta_"+str(beta)+"_gamma_"+str(gamma)+"_t_"+str(tt)+"_u_"+str(u)
            circs,meas,coefs,lengths = collect_energy_circs(alpha_k_A,phi_k_A,blocks,alpha_k_B,phi_k_B,blocks,alpha_k,phi_k,blocks)
            r = get_results(backend,circs, method = method,save_id_file = save_id_file)
            E_Ap, E_Bp, E_fp = three_energies_from_results(r,meas,coefs,lengths)
        else:
            #A
            save_id_file = "/Users/stenger/Documents/Research/Hubbard_symmetries/VQE/Job_id/jobA_"+str(k)+'_T_'+str(T)+"_A_"+str(A)+"_a_"+str(a)+"_c_"+str(c)+"_beta_"+str(beta)+"_gamma_"+str(gamma)+"_t_"+str(tt)+"_u_"+str(u)
            #print(tt)
            #print(save_id_file)
            circs_A, meas_A, coefs_A = collect_circs(alpha_k_A,phi_k_A,blocks)
            r_A = get_results(backend,circs_A, method = method,save_id_file = save_id_file)
            E_A = energy_from_results(r_A,meas_A,coefs_A)
            now = datetime.now().strftime("%H:%M:%S")
            user_messenger.publish({'k':k,'E_A':E_A,'time':now})
            #B
            save_id_file = "/Users/stenger/Documents/Research/Hubbard_symmetries/VQE/Job_id/jobB_"+str(k)+'_T_'+str(T)+"_A_"+str(A)+"_a_"+str(a)+"_c_"+str(c)+"_beta_"+str(beta)+"_gamma_"+str(gamma)+"_t_"+str(tt)+"_u_"+str(u)
            circs_B, meas_B, coefs_B = collect_circs(alpha_k_B,phi_k_B,blocks)
            r_B = get_results(backend,circs_B, method = method,save_id_file = save_id_file)
            E_B = energy_from_results(r_B,meas_B,coefs_B)
            now = datetime.now().strftime("%H:%M:%S")
            user_messenger.publish({'k':k,'E_B':E_B,'time':now})
            #f
            save_id_file = "/Users/stenger/Documents/Research/Hubbard_symmetries/VQE/Job_id/jobf_"+str(k)+'_T_'+str(T)+"_A_"+str(A)+"_a_"+str(a)+"_c_"+str(c)+"_beta_"+str(beta)+"_gamma_"+str(gamma)+"_t_"+str(tt)+"_u_"+str(u)
            circs_f, meas_f, coefs_f = collect_circs(alpha_k,phi_k,blocks)
            r_f = get_results(backend,circs_f, method = method,save_id_file = save_id_file)
            E_f = energy_from_results(r_f,meas_f,coefs_f)
            now = datetime.now().strftime("%H:%M:%S")
            user_messenger.publish({'k':k,'E_f':E_f,'time':now})

        
        #Calculate gradiant
        g = np.real((E_A-E_B)/(2*c_k)) 
        
        #Update phi
        g_k = g * Delta_k
        phi_k = phi_k - a_k * g_k
        
        #Update alpha
        g_ak = g * Delta_ak
        alpha_k = alpha_k - a_ak * g_ak
        norm = 1/np.sqrt(np.dot(alpha_k,alpha_k))
        alpha_k = norm*alpha_k

        
        #Print and save E
        #print(k,E_f)
        #print('k=',k,'c_k=',c_k,'a_k=',a_k,'g=',g,'E_A=',E_A,'E_B=',E_B,'E_f=',E_f)
        E_l.append(E_f)
        
        if hold == True:
            hold_k = {'E':E_f,'c':c_k,'a':a_k,'phi':phi_k,'Delta':Delta_k,'c_a':c_ak,'a_a':a_ak,'alpha':alpha_k,'Delta_a':Delta_ak}
            #np.save("/Users/stenger/Documents/Research/Hubbard_symmetries/VQE/Job_id/run_"+str(k)+'_T_'+str(T)+"_A_"+str(A)+"_a_"+str(a)+"_c_"+str(c)+"_beta_"+str(beta)+"_gamma_"+str(gamma)+"_t_"+str(tt)+"_u_"+str(u),[hold_k])
            hold_l.append(hold_k)
    if hold == True:
        return hold_l
    else:
        return E_l,phi_k
    
####### Main ########
def main(backend, user_messenger, **kwargs):
    k_max = kwargs.pop('k_max', 10)
    tt = kwargs.pop('tt')
    u = kwargs.pop('u')
    phi = kwargs.pop('phi')
    alpha = kwargs.pop('alpha')
    blocks = kwargs.pop('blocks')
    seeded = kwargs.pop('seeded',False)
    method = kwargs.pop('method','quantum')
    user_messenger.publish({"Starting program with k_max": k_max})
    out = SPSA(backend, user_messenger, k_max, tt, u, phi, alpha, blocks, method = method, group = False, hold = True)
    return out

Overwriting runtime_group_vqe.py


# Upload the program

In [15]:
import json

program_details = {
  "name": "Grouped_VQE",
  "description": "Performs VQE on Hamiltonian blocks so that the Hamitonian can be split into classical and quantum parts.  This program was written by Dr. John Stenger.  For help email me at jstenge2@gmail.com",
  "max_execution_time": 28800,
  "version": "1.0",
  "backend_requirements": {"min_num_qubits":  3},
  "parameters": [
    {"name": "k_max", "description": "The number of iterations for the VQE algorithm", "type": "int", "required": True}
  , {"name": "phi", "description": "The starting angles for the anzats circuit.  The form must be [classical block index][circuit depth index][qubit index]", "type": "nested list of dim 3", "required": True}
  , {"name": "alpha", "description": "The starting values of the wieghts of each block.  The form must be [classical block index]", "type": "nested list of dim 3", "required": True}
  ],
  "return_values": [
    {"name": "output", "description": "A collection of paramter values at each iteration", "type": "list of dictionaries"}
  ],
  "interim_results": [
    {"name": "k", "description": "Iteration number.", "type": "int"},
    {"name": "Ef", "description": "The energy at the end of the step", "type": "int"}
  ]
}

out_file = open("runtime_block_vqe_details.json", "w")
json.dump(program_details, out_file)
out_file.close()

in_file = open("runtime_block_vqe_details.json",)
details = json.load(in_file)
in_file.close()

print(details)

{'name': 'Grouped_VQE', 'description': 'Performs VQE on Hamiltonian blocks so that the Hamitonian can be split into classical and quantum parts.  This program was written by Dr. John Stenger.  For help email me at jstenge2@gmail.com', 'max_execution_time': 28800, 'version': '1.0', 'backend_requirements': {'min_num_qubits': 3}, 'parameters': [{'name': 'k_max', 'description': 'The number of iterations for the VQE algorithm', 'type': 'int', 'required': True}, {'name': 'phi', 'description': 'The starting angles for the anzats circuit.  The form must be [classical block index][circuit depth index][qubit index]', 'type': 'nested list of dim 3', 'required': True}, {'name': 'alpha', 'description': 'The starting values of the wieghts of each block.  The form must be [classical block index]', 'type': 'nested list of dim 3', 'required': True}], 'return_values': [{'name': 'output', 'description': 'A collection of paramter values at each iteration', 'type': 'list of dictionaries'}], 'interim_result

In [16]:
import os
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')  # Substitute with your provider.


ibmqfactory.load_account:WARNING:2022-04-05 09:14:19,688: Credentials are already in use. The existing account in the session will be replaced.


In [17]:
sample_program_json = os.path.join(os.getcwd(), "runtime_block_vqe_details.json")
sample_program_data = os.path.join(os.getcwd(), "runtime_group_vqe.py")

In [18]:
import os
from qiskit_ibm_runtime import IBMRuntimeService

service = IBMRuntimeService()

program_id = service.upload_program(data=sample_program_data, metadata=sample_program_json)
print(program_id)

grouped-vqe-7E5J23kZ08


# Deleting the program

In [ ]:
#Delete the program
program_id = "block-vqe-kd97bb6WkG"
provider.runtime.delete_program(program_id)

# Testing the program

Must run the first cell as a cell instead of creating a file.

In [6]:

from qiskit.opflow import (I, X, Y, Z)
from qiskit.opflow.primitive_ops import MatrixOp
from qiskit import quantum_info as qi
import numpy as np 
import copy

#H = np.load('Hamiltonian_Example.npy')
# I have -2.0 < u 2.0 in steps of 0.1
u=4.0
tt=-1
path = '/Users/stenger/Documents/Research/Hubbard_symmetries/VQE/'
H = np.load(path + 'H_mu_'+str(u)+"_t_"+str(tt)+'.npy')

# Function to add elements to a block so that it fits onto qubits
def complete_diagonal(b11):
    size = 2**np.ceil(np.log2(len(b11)))
    for v in b11:
        while len(v) < size:
            v.append(0.0)
    while len(b11) < size:
        i = len(b11)
        vnew = [0.0 for i in range(0,8)]
        vnew[i] = 10
        b11.append(vnew)
    return b11

def complete_off_diagonal(b11):
    size = 2**np.ceil(np.log2(len(b11)))
    for v in b11:
        while len(v) < size:
            v.append(0.0)
    while len(b11) < size:
        i = len(b11)
        vnew = [0.0 for i in range(0,8)]
        b11.append(vnew)
    return b11

import math
N = 4
S = 2
bL = math.comb(4,2)

blocks = {}
for bi in range(bL):
    for bj in range(bi,bL):
        bij = [[H[i + bL*bi][j + bL*bj] for j in range(0,bL)] for i in range(0,bL)]
        if np.amax(np.abs(bij)) > 10**(-5):
            if bi == bj:
                bij = complete_diagonal(bij)
            else:
                bij = complete_off_diagonal(bij)
            bij_pauli = MatrixOp(bij).to_pauli_op()
            blocks[str(bi) + ',' + str(bj)] = bij_pauli
        
Q = int(np.log2(len(blocks['0,0'].to_matrix())))

#Set alpha and phi to initial value

alpha = 1/np.sqrt(bL)*np.array([1 for b in range(bL)])

#number of blocks
T = 1
phi = []
for b in range(bL):
    phi_b = []
    for ti in range(T):
        phi_t = []
        for q in range(Q):
            phi_t.append(0)
        phi_b.append(phi_t)
    phi.append(phi_b)
    
from qiskit import IBMQ,transpile
from qiskit.tools.monitor import job_monitor
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')
backend = provider.backend.ibmq_qasm_simulator




ibmqfactory.load_account:WARNING:2022-04-04 13:55:01,062: Credentials are already in use. The existing account in the session will be replaced.


In [7]:
import runtime_group_vqe as test

In [8]:

from qiskit.providers.ibmq.runtime import UserMessenger
user_messenger = UserMessenger()

test.main(backend,user_messenger,k_max = 3, tt = tt, u = u, phi = phi, alpha = alpha, blocks = blocks, method = 'matrix')


{"Starting program with k_max": 3}
{"starting iteration": 0}
{"k": 0, "E_A": 4.786998020398677, "time": "13:55:04"}
{"k": 0, "E_B": 3.7446482513561152, "time": "13:55:04"}
{"k": 0, "E_f": 2.6666666666666683, "time": "13:55:05"}
{"starting iteration": 1}
{"k": 1, "E_A": 4.142375945805254, "time": "13:55:05"}
{"k": 1, "E_B": 3.6826976533989733, "time": "13:55:06"}
{"k": 1, "E_f": 2.724557902196793, "time": "13:55:06"}
{"starting iteration": 2}
{"k": 2, "E_A": 3.990340912563197, "time": "13:55:07"}
{"k": 2, "E_B": 1.3952096652273371, "time": "13:55:07"}
{"k": 2, "E_f": 2.745561497950097, "time": "13:55:08"}


[{'E': 2.6666666666666683,
  'c': 0.4,
  'a': 0.030878064824598282,
  'phi': array([[[ 0.04023218,  0.04023218,  0.04023218]],
  
         [[ 0.04023218, -0.04023218,  0.04023218]],
  
         [[-0.04023218,  0.04023218, -0.04023218]],
  
         [[ 0.04023218, -0.04023218,  0.04023218]],
  
         [[-0.04023218, -0.04023218, -0.04023218]],
  
         [[-0.04023218, -0.04023218, -0.04023218]]]),
  'Delta': array([[[-1, -1, -1]],
  
         [[-1,  1, -1]],
  
         [[ 1, -1,  1]],
  
         [[-1,  1, -1]],
  
         [[ 1,  1,  1]],
  
         [[ 1,  1,  1]]]),
  'c_a': 0.4,
  'a_a': 0.030878064824598282,
  'alpha': array([0.41983619, 0.41983619, 0.41983619, 0.41983619, 0.34451106,
         0.41983619]),
  'Delta_a': array([-1., -1., -1., -1.,  1., -1.])},
 {'E': 2.724557902196793,
  'c': 0.372954594574733,
  'a': 0.030342724499598534,
  'phi': array([[[ 0.021533  ,  0.021533  ,  0.05893136]],
  
         [[ 0.05893136, -0.021533  ,  0.021533  ]],
  
         [[-0.021533  ,